### Con que datos nos quedamos?

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('data/properati-AR-2017-08-01-properties-sell.csv')

In [3]:
data.columns

Index(['id', 'created_on', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [4]:
data.operation.value_counts() ## Son todos sell

sell    187482
Name: operation, dtype: int64

In [5]:
data.country_name.value_counts() ## son todos arg

Argentina    187482
Name: country_name, dtype: int64

In [6]:
data.drop(['id', 'operation','country_name', 'properati_url', 'image_thumbnail'], 1, inplace=True)
## el id no dice nada creo y las url tampoco

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187482 entries, 0 to 187481
Data columns (total 22 columns):
created_on                    187482 non-null object
property_type                 187482 non-null object
place_name                    187447 non-null object
place_with_parent_names       187482 non-null object
state_name                    187482 non-null object
geonames_id                   158318 non-null float64
lat-lon                       118761 non-null object
lat                           118761 non-null float64
lon                           118761 non-null float64
price                         154443 non-null float64
currency                      154441 non-null object
price_aprox_local_currency    154443 non-null float64
price_aprox_usd               154443 non-null float64
surface_total_in_m2           125027 non-null float64
surface_covered_in_m2         152420 non-null float64
price_usd_per_m2              105024 non-null float64
price_per_m2                  13

In [8]:
data.state_name.value_counts()

Capital Federal                 44152
Bs.As. G.B.A. Zona Norte        39888
Bs.As. G.B.A. Zona Oeste        22571
Buenos Aires Costa Atlántica    20694
Bs.As. G.B.A. Zona Sur          18135
Córdoba                         15777
Santa Fe                        13516
Buenos Aires Interior            4686
Río Negro                        1152
Neuquén                          1057
Mendoza                           927
Misiones                          817
Entre Ríos                        793
Tucumán                           715
Corrientes                        629
San Luis                          444
Salta                             372
Chubut                            329
La Pampa                          301
Catamarca                         126
Chaco                              95
San Juan                           86
Tierra Del Fuego                   69
Formosa                            66
Santa Cruz                         40
Jujuy                              32
La Rioja    

In [9]:
data = data[data['state_name'].str.contains('G.B.A') | data['state_name'].str.contains('Capital Federal')]
##El tp dice solo gba y caba

In [10]:
data.state_name.value_counts()

Capital Federal             44152
Bs.As. G.B.A. Zona Norte    39888
Bs.As. G.B.A. Zona Oeste    22571
Bs.As. G.B.A. Zona Sur      18135
Name: state_name, dtype: int64

In [11]:
data['created_on'] = pd.to_datetime(data['created_on']) ## conversion de tipo

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124746 entries, 0 to 187480
Data columns (total 22 columns):
created_on                    124746 non-null datetime64[ns]
property_type                 124746 non-null object
place_name                    124711 non-null object
place_with_parent_names       124746 non-null object
state_name                    124746 non-null object
geonames_id                   97866 non-null float64
lat-lon                       75504 non-null object
lat                           75504 non-null float64
lon                           75504 non-null float64
price                         110203 non-null float64
currency                      110202 non-null object
price_aprox_local_currency    110203 non-null float64
price_aprox_usd               110203 non-null float64
surface_total_in_m2           89385 non-null float64
surface_covered_in_m2         109180 non-null float64
price_usd_per_m2              79108 non-null float64
price_per_m2                  

In [13]:
data_wop = data[data['price'].isnull()]

In [14]:
data_wop.info()### Los datos sin precio no tienen ningun tipo de precio

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14543 entries, 28 to 187464
Data columns (total 22 columns):
created_on                    14543 non-null datetime64[ns]
property_type                 14543 non-null object
place_name                    14542 non-null object
place_with_parent_names       14543 non-null object
state_name                    14543 non-null object
geonames_id                   12167 non-null float64
lat-lon                       9318 non-null object
lat                           9318 non-null float64
lon                           9318 non-null float64
price                         0 non-null float64
currency                      0 non-null object
price_aprox_local_currency    0 non-null float64
price_aprox_usd               0 non-null float64
surface_total_in_m2           10011 non-null float64
surface_covered_in_m2         10304 non-null float64
price_usd_per_m2              0 non-null float64
price_per_m2                  0 non-null float64
floor         

In [15]:
data.dropna(axis=0,subset=['price'],inplace=True) ## drop de los datos sin precio

In [16]:
data.info()
## hay muchos datos con precio pero sin precios por m2 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110203 entries, 0 to 187480
Data columns (total 22 columns):
created_on                    110203 non-null datetime64[ns]
property_type                 110203 non-null object
place_name                    110169 non-null object
place_with_parent_names       110203 non-null object
state_name                    110203 non-null object
geonames_id                   85699 non-null float64
lat-lon                       66186 non-null object
lat                           66186 non-null float64
lon                           66186 non-null float64
price                         110203 non-null float64
currency                      110202 non-null object
price_aprox_local_currency    110203 non-null float64
price_aprox_usd               110203 non-null float64
surface_total_in_m2           79374 non-null float64
surface_covered_in_m2         98876 non-null float64
price_usd_per_m2              79108 non-null float64
price_per_m2                  9